# MÁSTER en Behavioral Data Science - **Módulo 9: Aprendizaje Profundo** - Notebook 4/4.
Autor: **Meysam Madadi**

Colaborador: Julio C. S. Jacques Junior

---

# **Los objetivos de este Jupyter notebook**
- Crear nuestro modelo para datos multimodales (texto e imagen).
- Predecir personalidad aparente a partir de datos multimodales.
- Visualizar los resultados.

## Cargando y descomprimiendo los datos, e importando las librerias requeridas

El número total de muestras de imágenes por video es 4. Podemos definir cuántas imágenes se utilizarán para entrenamiento/prueba. Para esto podríamos cambiar **"n\_frames"** en la siguiente celda por un valor en el rango [1..4].

- Todos los modelos se implementan y entrenan utilizando Keras.
- Reimplementamos la función **model.fit()** en una nueva clase, llamada **MyModel**, disponible en el archivo **myfunctions.py**. Esto se debe a algunos problemas técnicos que tuvimos, muy probablemente causados por las limitaciones de memoria de Colab.

In [1]:
# Download and unzip the data
!wget https://data.chalearnlap.cvc.uab.cat/Colab_2021/UBmasterBDS/data_final.zip
!unzip ./data_final.zip

# Download the file in which we have reimplemented Model class
!wget https://data.chalearnlap.cvc.uab.cat/Colab_2021/UBmasterBDS/myfunctions.py

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: data_final/validation/qJ6_xSx9kCM.002/annotation.npy  
  inflating: data_final/validation/qJ6_xSx9kCM.002/transcription_features.npy  
   creating: data_final/validation/QJcc95Y0XPw.002/
  inflating: data_final/validation/QJcc95Y0XPw.002/001.jpg  
  inflating: data_final/validation/QJcc95Y0XPw.002/115.jpg  
  inflating: data_final/validation/QJcc95Y0XPw.002/229.jpg  
  inflating: data_final/validation/QJcc95Y0XPw.002/343.jpg  
  inflating: data_final/validation/QJcc95Y0XPw.002/annotation.npy  
  inflating: data_final/validation/QJcc95Y0XPw.002/transcription_features.npy  
   creating: data_final/validation/qjU3GX3jgSY.002/
  inflating: data_final/validation/qjU3GX3jgSY.002/001.jpg  
  inflating: data_final/validation/qjU3GX3jgSY.002/115.jpg  
  inflating: data_final/validation/qjU3GX3jgSY.002/229.jpg  
  inflating: data_final/validation/qjU3GX3jgSY.002/343.jpg  
  inflating: data_final/validation/qjU3GX3jgSY.002/

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization, ReLU, LeakyReLU, LSTM, Reshape, GlobalMaxPooling1D, GlobalAveragePooling1D, Concatenate
import keras.backend as K
import numpy as np
import tensorflow.keras.applications as app
# from tensorflow.keras.models import Model
# import reimplemented Model class
from myfunctions import MyModel, reshape
from glob import glob
import cv2

n_frames = 4
if n_frames>4:
  print("n_frames must be between 1 and 4. It is set to 4.")
  n_frames = 4
elif n_frames<1:
  print("n_frames must be between 1 and 4. It is set to 1.")
  n_frames = 1

## **Definiendo nuestra clase "DataGenerator" para cargar los datos en lotes**

Nuestra clase "DataGenerator" se inicializa con varias variables de entrada:
- **data_list**, que es una lista de nombres de archivos de video para conjuntos de entrenamiento, validación o prueba,
- **root** es la ruta al conjunto de destino (es decir, entrenamiento, validación o prueba),
- **n_frames** indica el número de imágenes por vídeo,
- **is_sequence** indica si las características de transcripción se dan como una secuencia de tokens o como promedio,
- **batch_size** es el tamaño del lote,
- **shuffle** indica si la lista de datos está desordenada o no (es habitual desordenar las muestras para que el algoritmo de aprendizaje reciba un orden diferente de muestras en cada epoch).

Los datos devueltos son una lista con datos de imagen y texto con la forma de **(batch_size, n_frames, 224, 224, 3)** para imágenes y **(batch_size, 114, 768)** o **(batch_size, 768)** dependiendo de **is_sequence**.

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_list, root, n_frames=4, is_sequence=True, batch_size=16, shuffle=True):
        super().__init__()
        'Initialization'
        self.data_list = data_list
        self.root = root
        self.n_frames = n_frames
        self.is_sequence = is_sequence
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.mean = np.array([[[103.939, 116.779, 123.68]]], dtype=np.float32)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.data_list) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Find list of IDs
        data_list_temp = self.getdatalist(index)

        # Generate data
        X, y = self.__data_generation(data_list_temp)

        return X, y

    def getdatalist(self, index):
        'Get the list of data for one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        data_list_batch = [self.data_list[k] for k in indexes]

        return data_list_batch

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, data_list):
      'Generates data containing batch_size samples' # X_img : (batch_size, n_frames, 224, 224, 3,), X_txt : (batch_size, 768)
      X_img, X_txt, Y = [], [], []
      n_max = 114
      for f in data_list:
        text_features = np.load(open(self.root+f+'/transcription_features.npy', 'rb'))[0]
        if self.is_sequence:
          n_seq, n_feat = text_features.shape
          # if the number of tokens is smaller than the maximum number of tokens,...
          # we pad the features with zero to have a fixed length sequence
          if n_seq < n_max:
            text_features = np.concatenate([np.zeros((n_max - n_seq, n_feat)), text_features], axis=0)
        else:
          text_features = np.mean(text_features, axis=0)
        X_txt.append(text_features)

        files = sorted(glob(self.root+f+'/*.jpg'))
        imgs = []
        for i in range(self.n_frames):
          img = cv2.imread(files[i])
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) - self.mean
          imgs.append(img)
        X_img.append(imgs)

        # ['extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness']
        traits = np.load(open(self.root+f+'/annotation.npy', 'rb'))
        Y.append(traits)

      return [np.array(X_img, np.float32), np.array(X_txt, np.float32)], np.array(Y, np.float32)

## **Construyendo nuestro modelo de red neuronal**

- Para diseñar nuestra red multimodal, usamos el conocimiento que obtuvimos en los Jupyter Notebooks anteriores. Definimos dos entradas para características de imagen y texto que se le dan a la clase "MyModel" para construir el modelo. Las imágenes y las características textuales son procesadas por ResNet50 y LSTM respectivamente. Las formas de salida de cada uno son **(batch_size, 2048)** y **(batch_size, 768)**.
- Antes de fusionar las características de las dos modalidades, realizamos una capa de proyección sobre estas representaciones de características. Primero se aplica una capa Densa para reducir la dimensionalidad (definida por **projection_dims**) y luego se realiza una secuencia de bloques Densos residuales ("Activation+Dense+Dropout") definida por **num_projection_layers**.
- Estas capas de proyección, que incluyen la reducción de la dimensionalidad y las conexiones residuales, ayudan a equilibrar las características entre dos modalidades mientras se tiene un entrenamiento estable y menos sobreajustado.
- Finalmente, las representaciones proyectadas deben fusionarse. Una fusión simple pero efectiva es la concatenación, lo que significa apilar las dos características en un vector de características con forma **(batch_size, F1+F2)** donde F1 y F2 son el número de características para las modalidades después de las capas de proyección. Al final, se utiliza una regresión simple MLP para predecir los rasgos de personalidad.

In [ ]:
def project_embeddings(embeddings, num_projection_layers, projection_dims, dropout_rate):
    projected_embeddings = tf.keras.layers.Dense(units=projection_dims)(embeddings)
    for _ in range(num_projection_layers):
        x = tf.nn.gelu(projected_embeddings)
        x = tf.keras.layers.Dense(projection_dims)(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)
        x = tf.keras.layers.Add()([projected_embeddings, x])
        projected_embeddings = tf.keras.layers.LayerNormalization()(x)
    return projected_embeddings

In [ ]:
is_sequential = True

# Create model
def create_model(is_sequential):

    net_name = ['resnet50','ResNet50']

    # Select the corresponding network class
    mynet = getattr(getattr(app, net_name[0]), net_name[1])

    base_model = mynet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    input_img = tf.keras.Input(shape=(n_frames, 224, 224, 3))

    x = reshape(n_frames)(input_img)#(batch_size*n_frames, 224, 224, 3)

    x = base_model(x)#(batch_size*n_frames, 7, 7, 2048)

    # add a global spatial max pooling layer among all frames of a video after reshaping the features
    x = reshape(n_frames)(x)#(batch_size, n_frames*7*7, 2048)
    # x = GlobalMaxPooling1D()(x)#(batch_size, 2048)
    x = GlobalAveragePooling1D()(x)#(batch_size, 2048)
    x1 = project_embeddings(x, 1, 512, 0.3)#(batch_size, 512)

    # Process the textual data
    if is_sequential:
      input_txt = tf.keras.Input(shape=(114, 768))# (batch_size, 114, 768)

      x = LSTM(768, input_shape=(114, 768), activation=None, return_sequences=True)(input_txt)# (batch_size, 114, 768)
      x = Dropout(0.2)(x)

      x = LSTM(768, activation=None)(x)# (batch_size, 768)
      x2 = project_embeddings(x, 1, 512, 0.3)# (batch_size, 512)
    else:
      input_txt = tf.keras.Input(shape=(768,))# (batch_size, 768)

      x2 = project_embeddings(x, 1, 512, 0.3)# (batch_size, 512)

    x = Concatenate()([x1, x2])

    x = Dropout(0.5)(x)
    fc_1 = Dense(256, activation='relu')(x)
    predictions = Dense(5, activation='sigmoid')(fc_1)

    # this is the model we will train
    model = MyModel(inputs=(input_img, input_txt), outputs=predictions)

    return model

model = create_model(is_sequential)
# print(model.summary())
tf.keras.utils.plot_model(model, show_shapes=True)

## Leyendo la lista de archivos de entrenamiento, validación y prueba

In [ ]:
# Read the data lists
with open('train.txt', 'r') as f:
  train_list = f.readlines()
  for i in range(len(train_list)):
    train_list[i] = train_list[i].rsplit('\n',1)[0]
with open('validation.txt', 'r') as f:
  validation_list = f.readlines()
  for i in range(len(validation_list)):
    validation_list[i] = validation_list[i].rsplit('\n',1)[0]
with open('test.txt', 'r') as f:
  test_list = f.readlines()
  for i in range(len(test_list)):
    test_list[i] = test_list[i].rsplit('\n',1)[0]

## Entrenando y evaluando nuestro modelo

El modelo se entrena con el optimizador Adam con una tasa de aprendizaje de 1e-5 y una función de pérdida de error cuadrático medio (L2). El tamaño del lote es 32 y el modelo se entrena durante 20 epochs. Se define un "callback" para guardar el mejor modelo entrenado, en función del error absoluto medio observado en el conjunto de validación. Finalmente, se guarda el registro del historial en la ruta definida y se evalúa el modelo en el conjunto de pruebas.

In [ ]:
# Training
import gc
import random
import pickle

lr = 1e-5
batch_size = 32
n_epochs = 20
checkpoint = './best_model_multimodal.h5'
shuffle = True
verbose = 1

n_frames = 4  # Setting number of images per video to 4

# creating data generators to load the data
train_dg = DataGenerator(train_list, './data_final/train/', n_frames=n_frames, is_sequence=is_sequential, batch_size=batch_size, shuffle=shuffle)
validation_dg = DataGenerator(validation_list, './data_final/validation/', n_frames=n_frames, is_sequence=is_sequential, batch_size=batch_size, shuffle=False)
test_dg = DataGenerator(test_list, './data_final/test/', n_frames=n_frames, is_sequence=is_sequential, batch_size=batch_size, shuffle=False)

# defining the optimizer
model.compile(tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.MeanSquaredError(), metrics=['mae'])

history = model.fit(train_dg, validation_dg, epochs=n_epochs, verbose=verbose, checkpoint=checkpoint)

with open('./train_history_multimodal.pkl', 'wb') as handle:
  pickle.dump(history, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Ceating/building our model again, and loading the last checkpoint (best model)
model = create_model(is_sequential)
model.load_weights(checkpoint)
model.compile(tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.MeanSquaredError(), metrics=['mae'])

# Evaluate the trained model on the test set

# Some house keeping
gc.collect()
tf.keras.backend.clear_session()

print('Evaluating on the test set')

_loss = 0
_mae = 0
for step in range(test_dg.__len__()):
  # Load the batch
  X, Y = test_dg.__getitem__(step)
  X = [tf.convert_to_tensor(x, dtype=tf.float32) for x in X]
  Y = tf.convert_to_tensor(Y, dtype=tf.float32)

  # validate on one batch
  loss, mae = model.evaluate(X, Y, verbose = 0)

  _loss += loss
  _mae += mae
step += 1
print("The final mean absolute error is {0:.5f}\n".format(_mae/step))

## Visualización del historial de entrenamiento
- Pérdida de entrenamiento y validación, y MAE para cada epoch.

In [ ]:
# Visualization
from matplotlib import pyplot as plt

train_hist = pickle.load(open('./train_history_multimodal.pkl',"rb"))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 4))
fig.suptitle('Training history', fontsize=14, fontweight='bold')

ax1.plot(train_hist['loss'])
ax1.plot(train_hist['val_loss'])
ax1.set(xlabel='epoch', ylabel='Loss')
ax1.legend(['train', 'valid'], loc='upper right')

ax2.plot(train_hist['mae'])
ax2.plot(train_hist['val_mae'])
ax2.set(xlabel='epoch', ylabel='MAE')
ax2.legend(['train', 'valid'], loc='upper right')

## Visualizando una muestra de predicción

In [ ]:
# Download the original video transcriptions
!wget http://158.109.8.102/FirstImpressionsV2/train-transcription.zip
!unzip train-transcription.zip

from glob import glob
import cv2
import numpy as np
import pickle
from matplotlib import pyplot as plt

print('Predicting on the first batch of the test set')
# Load the batch
step = 0
X, Y = test_dg.__getitem__(step)

# validate on one batch
predictions = model.predict([tf.convert_to_tensor(x, dtype=tf.float32) for x in X])

# Visualize an example prediction
root = './data_final/test/'
sample_in_batch = 0 # You can increase this number up to the batch_size-1
vid_name = test_dg.getdatalist(step)[sample_in_batch]

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15, 4))
fig.suptitle('Sampled images from '+vid_name, fontsize=14, fontweight='bold')

# reading and showing images
files = sorted(glob(root+vid_name+'/*.jpg'))
for i in range(4):
  img = cv2.imread(files[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  eval('ax'+str(i+1)).imshow(img)

# reading and printing the transcription
trans = pickle.load(open('transcription_training.pkl','rb'))
print('\n\n\nTranscription is')
print(trans[vid_name+'.mp4'])

# reading and printing the personality labels
traits = np.load(open(root+vid_name+'/annotation.npy', 'rb'))
print('\nApparent personality labels are')
print('\t\textraversion\tneuroticism\tagreeableness\tconscientiousness\topenness')
print('Labels\t\t{0:0.2f}\t\t{1:0.2f}\t\t{2:0.2f}\t\t{3:0.2f}\t\t\t{4:0.2f}\t\t'.format(traits[0],traits[1],traits[2],traits[3],traits[4]))
print('Predictions\t{0:0.2f}\t\t{1:0.2f}\t\t{2:0.2f}\t\t{3:0.2f}\t\t\t{4:0.2f}\t\t'.format(predictions[sample_in_batch][0],predictions[sample_in_batch][1],predictions[sample_in_batch][2],predictions[sample_in_batch][3],predictions[sample_in_batch][4]))